Reference:  
https://zhuanlan.zhihu.com/p/42310942  NLP中的文本表示方法  
https://zhuanlan.zhihu.com/p/56382372  nlp中的词向量对比：word2vec/glove/fastText/elmo/GPT/bert  
http://www.hankcs.com/nlp/word2vec.html#respond  word2vec原理推导与代码分析

## NLP中的文本表示
- 基于ONE-HOT, TF-IDF, TEXTRANK等的BAG-OF-WORDS
- 主题模型: LSA(SVD), pLSA, LDA;
- 基于词向量的固定表征: word2vec, fastText, glove
- 基于词向量的动态表征: elmo, GPT, bert
### 离散表示
##### One-hot
词的表示(文本中的一个基元)给每个词编码一个索引,根据索引对每个词进行ont-hot表示.  
*特点*: 数据稀疏,维度灾难, 无法表达单词与单词之间的相似程度, 无法表达一词多义的情况
##### BAG-OF-WORDS
也叫做计数向量表示, 一种文档表示方法: 忽略文档的单词顺序和语法,句法等要素, 将其仅仅看作是若干个词汇的集合,且文档中每个词的出现都是相互独立的.
描述: 有一个文档集合D，里面一共有M个文档，而文档里面的所有单词提取出来后，一起构成一个包含N个单词的词典，利用Bag-of-words模型，每个文档都可以被表示成为一个N维向量，向量中的每个元素表示对应的单词在文档中出现的次数. 这样，就可以利用计算机来完成海量文档的分类过程。

##### Bi-gram和N-gram
文档表示(文本/文档/句子)与词袋模型类似, 将相邻的两个或者N个词编上索引. 
一种基于统计语言模型的算法。它的基本思想是将文本里面的内容按照字节进行大小为N的滑动窗口操作，形成了长度是N的字节片段序列。  
每一个字节片段称为gram，对所有gram的出现频度进行统计，并且按照事先设定好的阈值进行过滤，形成关键gram列表，也就是这个文本的向量特征空间，列表中的每一种gram就是一个特征向量维度。  
该模型基于这样一种假设，第N个词的出现只与前面N-1个词相关，而与其它任何词都不相关，整句的概率就是各个词出现概率的乘积。这些概率可以通过直接从语料中统计N个词同时出现的次数得到。常用的是二元的Bi-Gram和三元的Tri-Gram。  
*特点*: 引入了词的顺序, 造成词向量的急剧膨胀, N一般取3 最多不超过5
##### TF-IDF
对语料库中的一个文档进行表示: 需要统计一个文档中所有词语在语料库中的逆文档频率. 得到文档中每个词语的逆文档频率. (一般会去除停用词)  
*特点*: 使用词语的各个单词频率描述一篇文档.  
*用途*: 抽取文档关键词. 优点是简单快速, 结果也比较符合实际情况; 缺点是无法体现词的位置信息, 仅以词频衡量词的重要性也不够全面.  
*sklearn中的TfidfVectorizer生成TF-IDF特征*:
```python
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
                                   ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
```
##### 共现矩阵(Cocurrence matrix)
用于计算词的向量表示, 如两个词在一句话或者一篇文章中共同出现,通常我们选择一个距离窗口, 如果窗口宽度为2则认为当前词前后两个词的距离范围内出现的词为共现. 扫描所有文本,统计每个单词的共现词构造共现矩阵(对角矩阵). 对共现矩阵进行PCA或者SVD操作进行降维.  
*特点*: 使用稠密向量,计算量大
##### LSA
LSA（Latent Semantic Analysis）是一种比较早的count-based的词向量表征工具，它也是基于co-occurance matrix的，只不过采用了基于奇异值分解（SVD）的矩阵分解技术对大矩阵进行降维，而我们知道SVD的复杂度是很高的，所以它的计算代价比较大。还有一点是它对所有单词的统计权重都是一致的。
##### TextRank
Reference: https://blog.csdn.net/wotui1842/article/details/80351386

### 分布式表示
离散表示虽然能够进行词语或者文本的表示, 进而用于情感分析或者文本分类的人物. 但其不能表示词语间的相似程度或者词语间的类比关系. 因此使用分布式表示, 其主要思想是: **一个词的含义可以用它周围的词进行表示, 因为相同上下文语境的词往往会有相似的含义**  
#### 静态表征
##### NNLM
主要目的是为了搭建模型用于*根据前N-1个词预测第N个词的概率*, 其目标函数为:
$$ L(\theta) = \sum_tlogP(w_t|w_{t-n}, w_{t-n+1},\dots,w_{t-1})$$
使用长度为n-1的滑动窗口遍历整个语料库求和, 使得目标概率最大化,计算量正比于语料库大小,且预测的所有词概率总和应为1.
$$ \sum_{w\in\{vocabulary\}}P(w|w_{t-n+1},\dots,w_{t-1}) $$
NNLM 网络结构如下:  
<img src="./imgs/NNLM.jpg" width="480" height="320" align="center"/>  
图中Matrix C称为投影矩阵,把one-hot映射到稠密向量得到$C(W_{t-n+1}),\cdots,C(W_{t-1})$
隐藏层将稠密向量进行拼接,并使用tanh作为激活函数
隐层后面连接全连接层,使用softmax作为激活函数,预测词表中每个词出现的概率.
*特点*: 主要目的是为了搭建模型用于预测下一个可能的词, 词向量只是副产物. softmax的计算量太大.
#####  word2vec
word2ec有两种语言模型CBOW和Skip-gram他们的结构如下图所示:
<img src="./imgs/word2vec.png" width="480" height="320" align="center"/> 
具体的CBOW如下:  
<img src="./imgs/cbow.png" width="480" height="320" align="center"/> 
Skip-gram如下:  
<img src="./imgs/skip-gram.jpg" width="480" height="320" align="center"/> 
目标函数可简要概括为$$ J = \sum_{w\in corpus} P(w|context(w)) $$

和NNLM相比, word2vec:  
- 取消了隐藏层, 减少了计算量  
- 采用上下文滑窗而不是前文滑窗  
- 投影层使用简单的求和平均,而不是拼接向量  
可以看到，取消隐藏层，投影层求和平均都可以一定程度上减少计算量，但输出层的数量在那里，比如语料库有500W个词那么隐藏层就要对500W个神经元进行全连接计算，这依然需要庞大的计算量。因此,word2vec又提出了两种优化方法:  
- 使用hierarchical softmax和Negative sampling对输出层进行优化  
- hierarchical softmax  
    霍夫曼树是一棵特殊的二叉树，了解霍夫曼树之前先给出几个定义：
    路径长度：在二叉树路径上的分支数目，其等于路径上结点数-1  
    结点的权：给树的每个结点赋予一个非负的值
    结点的带权路径长度：根结点到该结点之间的路径长度与该节点权的乘积 
    树的带权路径长度：所有叶子节点的带权路径长度之和
    霍夫曼树的定义为：在权为$w_1,w_2,\cdots,w_n$的$n$个叶子结点所构成的所有二叉树中，带权路径长度最小的二叉树称为最优二叉树或霍夫曼树, 可以看出，结点的权越小，其离树的根结点越远。
word2vec算法利用霍夫曼树，将平铺型softmax压缩成层级softmax，不再使用全连接。具体做法是根据文本的词频统计，将词频赋给结点的权。在霍夫曼树中，叶子结点是待预测的所有词，在每个子结点处，用sigmoid激活后得到往左走的概率p，往右走的概率为1-p。最终训练的目标是最大化叶子结点处预测词的概率。    
- Negative sampling  
 负例采样的想法比较简单，假如有m个待预测的词，每次预测一个正样本词，其他的m-1个词均为负样本。一方面正负样本数差别太大，另一方面负样本中可能有很多不常用或者的词预测时概率基本为0，我们不想在计算它们的概率上面消耗资源。  
比如现在待预测词有100W个，正常情况下我们分类的全连接层需要100W个神经元，我们可以根据词语的出现频率进行负例采样，一个正样本加上采样出的比如说999个负样本，组成1000个新的分类全连接层。  
采样尽量保持了跟原样本一样的分布，具体做法是将$[0,1]$区间均分108份，然后根据词出现在语料库中的次数赋予每个词不同的份额。
$$len(w) = \frac{counter(w)}{\sum_{u \in D}counter(u)}$$
然后在$[0,1]$区间掷骰子，落在哪个区间内就采样哪个样本。实际上，最终效果证明上式中取$counter(w)$的$\frac{3}{4}$次方效果最好，所以在应用中也是这么做的。
##### fastText  
fasttext的模型与CBOW类似，实际上，fasttext的确是由CBOW演变而来的。CBOW预测上下文的中间词，fasttext预测文本标签。与word2vec算法的衍生物相同，稠密词向量也是在训练神经网络的过程中得到的。fasttext的结构图如下:
<img src="./imgs/fasttext.png" width="480" height="320" align="center"/>
fasttext的输入是一段词的序列，即一篇文章或一句话，输出是这段词序列属于某个类别的概率，所以fasttext是用来做文本分类任务的.
fasttext中采用层级softmax做分类，这与CBOW相同。fasttext算法中还考虑了词的顺序问题，即采用N-gram，与之前介绍离散表示时的做法相同。如：今天天气非常不错，Bi-gram的表示就是：今天、天天、天气、气非、非常、常不、不错。fasttext做文本分类对文本的存储方式有要求：
\__label__1, It is a nice day.
\__label__2, I am fine,thank you.
\__label__3, I like play football.

其中的__label__为实际类别的前缀，也可以自己定义。fasttext有python实现：https://github.com/salestock/fastText.py
```python
classifier = fasttext.supervised(input_file, output, label_prefix='__label__')
result = classifier.test(test_file)
print(result.precision,result.recall)
```
其中，input_file是已经按上面的格式要求做好的训练集txt，output后缀为.model，是我们保存的二进制文件，label_prefix可以自定义我们的类别前缀。
##### glove
参考: http://www.fanyeong.com/2018/02/19/glove-in-detail/
全称叫Global Vectors for Word Representation，它是一个基于全局词频统计（count-based & overall statistics）的词表征（word representation）工具，它可以把一个单词表达成一个由实数组成的向量，这些向量捕捉到了单词之间一些语义特性，比如相似性（similarity）、类比性（analogy）等。我们通过对向量的运算，比如欧几里得距离或者cosine相似度，可以计算出两个单词之间的语义相似性。  
实现分为以下三步:
- 根据语料库（corpus）构建一个共现矩阵（Co-ocurrence Matrix)X(什么是共现矩阵)，矩阵中的每一个元素$X_{ij}$代表单词$i$和上下文单词$j$在特定大小的上下文窗口（context window）内共同出现的次数。一般而言，这个次数的最小单位是1，但是GloVe不这么认为：它根据两个单词在上下文窗口的距离$d$，提出了一个衰减函数(decreasing weighting):$decay = \frac{1}{d}$用于计算权重，也就是说距离越远的两个单词所占总计数（total count）的权重越小。
- 构建词向量（Word Vector）和共现矩阵（Co-ocurrence Matrix）之间的近似关系，论文的作者提出以下的公式可以近似地表达两者之间的关系：
$$ w_i^T\hat w_j + b_i + \hat b_j = log(X_{ij})$$
其中$w_i^T$和$\hat w_j$是我们最终要求解的词向量; $b_i, b_j$分别是两个词向量的bias term.
TODO: 为什么要使用这个公式, 为什么要构造两个词向量$w_i^T$和$\hat w_j$?
- 构造损失函数:$$J = \sum_{i,j=1}^Vf(X_{ij})(w_i^T\hat w_j + b_i + \hat b_j - log(X_{ij}))^2$$
损失函数的基本形式就是最简单的MSE(mean square loss), 只不过在此基础上加了一个权重函数$f(X_{ij})$. 这个函数有什么作用?  
在一个语料库中, 有很多单词一起出现的次数较多, 我们希望:  
    - 这些单词的权重大于那些很少一起出现的单词(rare co-occurrences)
    - 这个权重不会过大(overweighted), 当达到一定程度之后就不再增加
    - 如果两个单词没有一起出现, 即$X_{ij}=0$, 那么,他们不应该参与到loss function的计算当中去, 也就是$f(x)$要满足$f(0)=0$  
    作者采用了:  
    <img src="./imgs/glove_weight.png">
    函数图像如下图所示:  
    <img src="./imgs/glove-weight-function.jpg">  
Glove如何训练的:   
虽然很多人声称GloVe是一种无监督（unsupervised learing）的学习方式(因为它确实不需要人工标注label).  
但其实它还是有label的，这个label就是公式2中的$log(X_{ij})$，而公式2中的向量$w$和$\hat w$就是要不断更新/学习的参数，所以本质上它的训练方式跟监督学习的训练方法没什么不一样，都是基于梯度下降的.  
具体地，这篇论文里的实验是这么做的:  
采用了AdaGrad的梯度下降算法，对矩阵$X$中的所有非零元素进行随机采样，学习曲率（learning rate）设为0.05，在vector size小于300的情况下迭代了50次，其他大小的vectors上迭代了100次，直至收敛。最终学习得到的是两个vector是$w$和$\hat w$. 因为$X$是对称的（symmetric），所以从原理上讲$w$和$\hat w$也是对称的，他们唯一的区别是初始化的值不一样，而导致最终的值不一样。所以这两者其实是等价的，都可以当成最终的结果来使用。但是为了提高鲁棒性，我们最终会选择两者之和$w+\hat w$作为最终的vector（两者的初始化不同相当于加了不同的随机噪声，所以能提高鲁棒性）。在训练了400亿个token组成的语料后，得到的实验结果如下图所示:  
<img src="./imgs/glove-result.jpg">
图中表明,实验一共采用了三个指标:语义准确度, 语法准确度和总体准确度. 最终结果显示, Vector Dimension在300时性能达到最佳, Context window size 在6~10之间.  
特点: 克服了LSA计算量大, 所有单词的统计权重都一样的缺点, 克服了word2vec没有充分利用所有语料的缺点. 从论文中的实验结果来看, glove性能远超word2vec, 但也有人说glove和word2vec实际表现差不多.  
补充阅读: https://zhuanlan.zhihu.com/p/42073620

#### 动态表征  
静态表征中对于每一个单词都有唯一的一个embedding表示, 对于多义词显然这种做法不符合直觉,实际使用过程中, 静态词向量对于多义词的效果也确实不好(谁通过什么实验证明的?). 在词的分布式表示中, 假设词的意思是由其上下文来表示的, 动态表征的做法是我们只预训练language model，而word embedding 是通过输入的句子实时输出的, 这样一个单词的意思就和上下文关联上了, 很大程度上缓解了歧义的发生.
动态表征已有的模型有:  
- ELMO
- GPT
- bert
##### ELMO  
Reference:https://www.cnblogs.com/jiangxinyang/p/10060887.html  

ELMO的架构如下图所示:
<img src='./imgs/elmo.jpg'>
Source: https://zhuanlan.zhihu.com/p/51679783  

给定一个含有$N$个tokens的序列$t_1,t_2,\cdots,t_N$, 其前向表示为:
$$p(t_1,t_2,\cdots,t_N) = \prod_{k=1}^Np(t_k|t_1,t_2,\cdots,t_{k-1})$$
反向表示为:$$p(t_1,t_2,\cdots,t_N) = \prod_{k=1}^Np(t_k|t_{k+1},t_{k+2},\cdots,t_N)$$
从联合概率公式来看,是一个典型的语言模型，前向利用上文来预测下文，后向利用下文来预测上文。假设输入的token是$x_k^{LM}$,在每一个位置$k$, 每一层LSTM上都输出相应的context-dependent的表征$\stackrel{\rightarrow LM}{Y_{k,j}}$, 这里$j=1,2,\cdots,L, L$表示LSTM的层数, 顶层的输出$\stackrel{\rightarrow LM}{Y_{k,L}}$, 通过softmax来预测下一个$token_{k+1}$.对数似然函数表示如下:
$$\sum_{k=1}^N(logp(t_k|t_1,\cdots,t_{k-1}; \Theta_x, \stackrel{\rightarrow}{\Theta_{LSTM}}, \Theta_S) + logp(t_k|t_{k+1},\cdots,t_N; \Theta_x, \stackrel{\leftarrow}{\Theta_{LSTM}}, \Theta_S))$$
ELMO模型不同于之前的其它模型,只用最后一层的输出值来作为word embedding的值, 而是用所有层的输出值的线性组合来表示word embedding的值, 对于每一个token, 一个L层的biLM要计算出$2L+1$个表征:
$$ R_k = \{x_k^{LM},\stackrel{\rightarrow LM}{Y_{k,j}},\stackrel{\leftarrow LM}{Y_{k,j}}| j=1,\cdots,L\}= \{h_{k,j}^{LM}| j=1,\cdots,L\}$$  
上式中$X_k^{LM}$就等于$h_{k,j}^{LM}$表示的是token层的值. 在下游任务中会把$R_k$压缩成一个向量:
$$ELMO_k^{task} = E(R_k;\Theta^task) = \gamma^{task}\sum_{j=0}^Ls_j^{task}h_{k,j}^{LM}$$
其中, $s_j^{task}$是softmax标准化权重, $\gamma^{task}$是缩放系数,允许任务模型区缩放整个ELMO向量. 
ELMo的使用主要有三步:
- 在大的语料库上预训练biLM模型. 模型由两层bi-LSTM组成, 模型之间用residual connection 连接起来. 作者认为, 低层的bi-LSTM层能提取语料的语法信息, 高层的bi-LSTM能提取语料的语义信息.
- 在训练语料(去除标签)中, fine-tuning预训练好的bi-LSTM模型. 这一步可以看作是bi-LSTM的domain transfer.
- 利用ELMO 产生的word embedding来作为任务的输入, 有时也可以即在输入时加入, 又在输出时加入.  
TODO: https://petrlorenc.github.io/ELMO/  
ELMO预训练的语言模型结构和下游的NLP任务的模型经常会不一致, 因此, 做迁移学习时经常不能将两者的模型直接进行融合. 因此, 2018年提出了一个新方法GPT.
##### GPT  
Reference: https://blog.csdn.net/linchuhai/article/details/97274228  
GPT(Generate pre-training)的基本思想是,先利用大量未标注的语料预训练一个语言模型, 接着对预训练好的语言模型进行微改, 将其迁移到各种有监督的NLP任务中, 并对参数进行fine-tuning.  

主要是针对文本分类和标注性任务, 对于生成任务, 如机器翻译等, 则其结构也没法进行很好的迁移.
##### bert  